In [67]:
from pathlib import Path
import pandas as pd
import altair as alt
import seaborn as sns

In [68]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [69]:
data_dir = Path.cwd().parent.parent / "results" / "test-01"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,190.0,0,NaN,NaN,NaN
1,1,True,190.0,0,NaN,NaN,NaN
2,2,False,190.0,2,40.0,Tanker,Ascent
3,2,False,190.0,3,190.0,Tanker,PropTransfer
4,4,False,550.0,2,40.0,Tanker,Ascent


#### Summary Statistics

In [70]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.5775775775775776 = 57.758 %



#### Mission Duration

In [71]:
df = mc_results.groupby("replicant", as_index=False).first()[["replicant", "duration"]]

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[0, 4000], step=250)),
    y='count()',
).properties(
    width = 800
)

alt.Chart(...)

Compute the Emprical CDF

In [72]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(x = "duration", y = "prob")
    return fig


In [73]:
plot_ECDF(df)

alt.Chart(...)